# This Notebook contains examples on simulated data

In [ ]:
import sys
# import os,inspect
# currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
# parentdir = os.path.dirname(currentdir)
# parentdir = os.path.dirname(parentdir)
parentdir = "C:\\Users\\cosim\\PycharmProjects\\DDFM"
sys.path.insert(0,parentdir)
import random
import numpy as np
import pandas as pd
from synthetic_dgp.simulate import SIMULATE
from statsmodels.tsa.statespace.dynamic_factor_mq import DynamicFactorMQ as DFM
from models.ddfm import DDFM
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# seed setting
seed = 123
random.seed(seed)
np.random.seed(seed)
# set poly degree, if 1 then linear and the encoder structure is modified to linear single layer
poly_degree = 2
# init results storage
results_dfm = np.zeros(2)
results_ddfm = np.zeros(1)
results_ddfm_nnlin_dec = np.zeros(2)
# params sim data
n_obs = 150
r = 3
portion_missings = 0.2
# simulate data
sim_40 = SIMULATE(seed=seed, n=40, r=r, poly_degree=poly_degree)
sim_150 = SIMULATE(seed=seed, n=150, r=r, poly_degree=poly_degree)

## Simulate data 40

In [ ]:
x = sim_40.simulate(n_obs, portion_missings=portion_missings)
r_true = sim_40.f.shape[1]
if poly_degree == 1:
    # For linear to set the following
    structure_encoder = (r_true,)
else:
    structure_encoder = (r_true * 6, r_true * 4, r_true * 2, r_true)
# for nonlinear decoder   
structure_encoder_nnlin = (r_true, r * 9, r * 3, r)
structure_decoder_nnlin = (r * 3, r * 9, r_true)

In [ ]:
%%time
# estimate dfm
dyn_fact_mdl = DFM(pd.DataFrame(x), factors=min(r_true, x.shape[1]), factor_orders=1)
res_dyn_fact_mdl = dyn_fact_mdl.fit(disp=1000, maxiter=10000)

In [ ]:
results_dfm[0] = sim_40.evaluate(res_dyn_fact_mdl.factors.smoothed.values, f_true=sim_40.f)
results_dfm[1] = sim_40.evaluate(res_dyn_fact_mdl.factors.filtered.values, f_true=sim_40.f)
print(results_dfm)

In [ ]:
%%time
# estimate ddfm (with linear decoder)
deep_dyn_fact_mdl = DDFM(pd.DataFrame(x), structure_encoder=structure_encoder, factor_oder=1,
                             use_bias=False, link='relu', max_iter=1000)
deep_dyn_fact_mdl.fit()

In [ ]:
results_ddfm[0] = sim_40.evaluate(np.mean(deep_dyn_fact_mdl.factors, axis=0), f_true=sim_40.f)
print(results_ddfm)

In [ ]:
%%time
# estimate ddfm (with nonlinear decoder)
deep_dyn_fact_mdl_nnlin = DDFM(pd.DataFrame(x), 
                               structure_encoder=structure_encoder_nnlin, 
                               factor_oder=1,
                               structure_decoder=structure_decoder_nnlin,
                               use_bias=False, link='relu', max_iter=1000)
deep_dyn_fact_mdl_nnlin.fit()

In [ ]:
results_ddfm_nnlin_dec[0] = sim_40.evaluate(np.mean(deep_dyn_fact_mdl_nnlin.last_neurons, axis=0), f_true=sim_40.f)
results_ddfm_nnlin_dec[1] = sim_40.evaluate(np.mean(deep_dyn_fact_mdl_nnlin.factors, axis=0), f_true=sim_40.linear_f)
print("last neurons", results_ddfm_nnlin_dec[0])
print("code layer against linear f", results_ddfm_nnlin_dec[1])


## Simulate data 150

In [ ]:
x = sim_150.simulate(n_obs, portion_missings=portion_missings)
r_true = sim_150.f.shape[1]
if poly_degree == 1:
    # For linear to set the following
    structure_encoder = (r_true,)
else:
    structure_encoder = (r_true * 6, r_true * 4, r_true * 2, r_true)
# for nonlinear decoder   
structure_encoder_nnlin = (r_true, r * 9, r * 3, r)
structure_decoder_nnlin = (r * 3, r * 9, r_true)

In [ ]:
%%time
# estimate dfm
dyn_fact_mdl = DFM(pd.DataFrame(x), factors=min(r_true, x.shape[1]), factor_orders=1)
res_dyn_fact_mdl = dyn_fact_mdl.fit(disp=1000, maxiter=1000)

In [ ]:
results_dfm[0] = sim_150.evaluate(res_dyn_fact_mdl.factors.smoothed.values, f_true=sim_150.f)
results_dfm[1] = sim_150.evaluate(res_dyn_fact_mdl.factors.filtered.values, f_true=sim_150.f)
print(results_dfm)

In [ ]:
%%time
# estimate ddfm
deep_dyn_fact_mdl = DDFM(pd.DataFrame(x), structure_encoder=structure_encoder, factor_oder=1,
                             use_bias=False, link='relu',
                         max_iter=1000)
deep_dyn_fact_mdl.fit()

In [ ]:
results_ddfm[0] = sim_150.evaluate(np.mean(deep_dyn_fact_mdl.factors, axis=0), f_true=sim_150.f)
print(results_ddfm)

In [ ]:
%%time
# estimate ddfm (with nonlinear decoder)
deep_dyn_fact_mdl_nnlin = DDFM(pd.DataFrame(x), 
                               structure_encoder=structure_encoder_nnlin, 
                               factor_oder=1,
                               structure_decoder=structure_decoder_nnlin,
                               use_bias=False, link='relu', max_iter=1000)
deep_dyn_fact_mdl_nnlin.fit()

In [ ]:
results_ddfm_nnlin_dec[0] = sim_40.evaluate(np.mean(deep_dyn_fact_mdl_nnlin.last_neurons, axis=0), f_true=sim_40.f)
results_ddfm_nnlin_dec[1] = sim_40.evaluate(np.mean(deep_dyn_fact_mdl_nnlin.factors, axis=0), f_true=sim_40.linear_f)
print("last neurons", results_ddfm_nnlin_dec[0])
print("code layer against linear f", results_ddfm_nnlin_dec[1])